In [31]:
import logging
import os
import re
from typing import List

import openai
from dotenv import load_dotenv
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMChain, RetrievalQAWithSourcesChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from llama_index import Prompt, StorageContext, load_index_from_storage
from pydantic import BaseModel, Field

In [32]:
openai_api_key = os.getenv("OPENAI_API_KEY") 
openai.api_key = openai_api_key

# Search
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# Llama Index

In [9]:
qa_template = Prompt(
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact.\n"
    "Always state where the information was found, and if possible where more information can be accessed.\n"
    "Given this information, please provide me with an answer to the following question:\n"
    "\n{query_str}\n"
)

# LLM
llm = ChatOpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=False, text_qa_template=qa_template, similarity_top_k=5
)

In [16]:
response = query_engine.query('What do I have to do if I know I will become unemployed in the next month?')

According to the provided context information from page 13 of the document "merkblatt-fuer-arbeitslose_ba036520.pdf", if you know that you will become unemployed in the next month, you are required to make an early notification of your job search to the Agentur für Arbeit (Employment Agency) no later than three months before the termination of your employment or training relationship. If you become aware of the termination date less than three months before it occurs, you must notify the agency within three days of becoming aware of the termination date. This notification can be done online, in person, by phone, or in writing. Failure to make this early notification may result in a one-week waiting period (sperrzeit) for receiving unemployment benefits. 

Source: "merkblatt-fuer-arbeitslose_ba036520.pdf", page 13. More information can be found on the website of the Bundesagentur für Arbeit (Federal Employment Agency) at www.arbeitsagentur.de.
{'56da56ac-29d9-41bb-8647-45a02bfb7416': {'

In [289]:
def parse_index_response(index_response):
    response_dict = {}
    response_dict['output_text'] = index_response.response
    response_dict['Documents'] = index_response.metadata
    return response_dict

parse_index_response(response)

{'output_text': 'According to the provided context information from page 13 of the document "merkblatt-fuer-arbeitslose_ba036520.pdf", if you know that you will become unemployed in the next month, you are required to make an early notification of your job search to the Agentur für Arbeit (Employment Agency) no later than three months before the termination of your employment or training relationship. If you become aware of the termination date less than three months before it occurs, you must notify the agency within three days of becoming aware of the termination date. This notification can be done online, in person, by phone, or in writing. Failure to make this early notification may result in a one-week waiting period (sperrzeit) for receiving unemployment benefits. \n\nSource: "merkblatt-fuer-arbeitslose_ba036520.pdf", page 13. More information can be found on the website of the Bundesagentur für Arbeit (Federal Employment Agency) at www.arbeitsagentur.de.',
 'Documents': {'56da56

# Retriever

In [359]:
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search 
    results. Generate FIVE Google search queries that are similar to
    this question. The output should be a numbered list of questions and each
    should have a question mark at the end: {question}""",
)


class LineList(BaseModel):
    """List of questions."""

    lines: List[str] = Field(description="Questions")


class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""

    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)

# LLM
llm = ChatOpenAI(temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=search_prompt,
    output_parser=QuestionListOutputParser(),
)


# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
)

# Search
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain,
    search=search,
)

def web_chain(user_input):
    docs = web_research_retriever_llm_chain.get_relevant_documents(user_input)
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain( {"input_documents": docs, "question": user_input}, return_only_outputs=True ), docs

In [ ]:
def parse_websearch_output(websearch_output):
    '''creates a dictionary of the websearch tool that resembles the ouput of 
    the output llama index tool'''
    
    wsearch_dict = {}
    wsearch_dict["output_text"] = websearch_output[0]["output_text"]
    wsearch_dict["Documents"] = {}

    # enumerate all documents and create sub-dictionaries
    for i, doc in enumerate(websearch_output[1]):
        document_info = {
            f"Document {i}": {
                "title": doc.metadata["title"],
                "content": doc.page_content,
                "source": doc.metadata["source"],
            }
        }
        
        # Append the new document to the existing documents dictionary
        wsearch_dict["Documents"].update(document_info)

    return wsearch_dict

In [360]:
rresponse = parse_websearch_output(web_chain("Where can I find information about the duties of landlords in Berlin Germany?"))

INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Where can I find information about the duties of landlords in Berlin Germany?', 'text': LineList(lines=['1. What are the responsibilities of landlords in Berlin, Germany?\n', '2. How can I learn about the obligations of landlords in Berlin, Germany?\n', '3. Where can I find details about the duties of landlords in Berlin, Germany?\n', '4. What information is available on the roles of landlords in Berlin, Germany?\n'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the responsibilities of landlords in Berlin, Germany?\n', '2. How can I learn about the obligations of landlords in Berlin, Germany?\n', '3. Where can I find details about the duties of landlords in Berlin, Germany?\n', '4. What information is available on the roles of landlords in Berlin, Germany?\n']
INFO:langchain.retrie

In [361]:
rresponse

{'output_text': "You can find information about the duties of landlords in Berlin, Germany in the relevant laws and regulations. Specifically, you can refer to the German Civil Code (Bürgerliches Gesetzbuch) and the Berlin Tenancy Act (Berliner Mietspiegel). These laws outline the rights and obligations of landlords and tenants in Berlin, including regulations regarding rent, maintenance, repairs, and termination of tenancy. Additionally, you may also find information on the website of the Berlin State Government or the Berlin Tenants' Association (Berliner Mieterverein).",
 'Documents': {'Document 0': {'title': 'Basic Law for the Federal Republic of Germany',
   'content': '(6) In accordance with the internal allocation of competencies and\nresponsibilities, the Federation and the _L ander_ shall bear the costs\nentailed by a violation of obligations incumbent on Germany under\nsupranational or international law. In cases of financial corrections by the\nEuropean Union with effect tra

In [ ]:
# Split the string based on the pattern 'Document(page_content='
split_output = result.split("Document(")

split_output

["({'output_text': 'The provided context does not specifically mention the duties of landlords in Berlin, Germany. Therefore, I do not have the information to answer your question.'}, [",
 "page_content='(6) In accordance with the internal allocation of competencies and\\nresponsibilities, the Federation and the _L ander_ shall bear the costs\\nentailed by a violation of obligations incumbent on Germany under\\nsupranational or international law. In cases of financial corrections by the\\nEuropean Union with effect transcending one specific _Land_ , the Federation\\nand the _L ander_ shall bear such costs at a ratio of 15 to 85. In such cases,\\nthe _L ander_ as a whole shall be responsible in solidarity for 35 per cent of\\nthe total burden according to a general formula; 50 per cent of the total\\nburden shall be borne by those _L ander_ which have caused the encumbrance,\\nadjusted to the size of the amount of the financial means received. Details\\nshall be regulated by a federal l

# Parse Websearch Output

In [356]:

def parse_websearch_output(websearch_output):
    '''creates a dictionary of the websearch tool that resembles the ouput of 
    the output llama index tool'''
    
    wsearch_dict = {}
    wsearch_dict["output_text"] = rresponse[0]["output_text"]
    wsearch_dict["Documents"] = {}

    # enumerate all documents and create sub-dictionaries
    for i, doc in enumerate(rresponse[1]):
        document_info = {
            f"Document {i}": {
                "title": doc.metadata["title"],
                "content": doc.page_content,
                "source": doc.metadata["source"],
            }
        }
        
        # Append the new document to the existing documents dictionary
        wsearch_dict["Documents"].update(document_info)

    return wsearch_dict

In [315]:
print(rresponse[0]['output_text'])

for i, doc in enumerate(rresponse[1]):
    print(f'Document: {i+1}')
    print('')
    print(doc.page_content)
    
    

You can find information about the duties of landlords in Berlin, Germany in the relevant laws and regulations. Specifically, you can refer to the German Civil Code (Bürgerliches Gesetzbuch) and the Berlin Tenancy Act (Berliner Mietspiegel). These laws outline the rights and obligations of landlords and tenants in Berlin, including regulations regarding rent, maintenance, repairs, and termination of tenancy. Additionally, you may also find information on the website of the Berlin State Government or the Berlin Tenants' Association (Berliner Mieterverein).
Document: 1

(6) In accordance with the internal allocation of competencies and
responsibilities, the Federation and the _L ander_ shall bear the costs
entailed by a violation of obligations incumbent on Germany under
supranational or international law. In cases of financial corrections by the
European Union with effect transcending one specific _Land_ , the Federation
and the _L ander_ shall bear such costs at a ratio of 15 to 85. In

# Parse Agent output

In [295]:
def parse_agent_output(agent_output):
    result_dict = {}
    result_dict['output_text'] = re.search(r"'output_text': '([^']*)'", agent_output).group(1)
    result_dict['Documents'] = {}

    documents = re.findall(r"Document\(page_content='(.+?)', metadata={(.+?)}\)", agent_output)

    for i, doc in enumerate(documents):
        doc_dict = {}
        
        match_title = re.search(r"'title': '([^']*)'", doc[1])
        if match_title:
            doc_dict['title'] = match_title.group(1)
            
        page_content = doc[0]
        clean_content = page_content.replace('\\n', ' ').strip()
        doc_dict['content'] = clean_content
        
        match_source = re.search(r"'source': '([^']*)'", doc[1])
        if match_source:
            doc_dict['source'] = match_source.group(1)
            
        result_dict['Source'][f'Source: {i}'] = doc_dict
        
    return result_dict
            
# function printing the agent output     
def print_agent_output(agent_output):
    # parse agent_output converts agent output (str) in a dicitonary
    result_dict = parse_agent_output(agent_output)
    
    print(result_dict.get('output_text'))
    print("\n")

    # Printing each entry in the nested dictionary separately
    for key, value in result_dict.items():
        if isinstance(value, dict):
                # Printing entries of the sub-dictionary
                for sub_key, sub_value in value.items():
                    print(f"{sub_key}")
                    print(f"{sub_value['title']}")
                    print(f"{sub_value['content']}")
                    print(f"{sub_value['source']}")
                    print("\n")
                    
print_agent_output(rresponse)

TypeError: expected string or bytes-like object

# Agent 

In [271]:

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="Useful for when you want to answer questions about work and unemployment.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(web_chain(q)),
        description="Useful for when you want to answer more general questions concerning laws and administration in Berlin, Germany.",
        return_direct=True,
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

result = agent_executor.run(
    # input="Where can I find information about the duties of landlords?"
    input="What do I have to do if I know I will be unemployed by next month?"
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: LlamaIndex
Action Input: What to do if unemployed next month


Observation: If you are facing unemployment next month, you should take the following steps:

1. Arbeitsuchendmeldung: You should inform the Agentur für Arbeit (Federal Employment Agency) about your impending unemployment by making an "Arbeitsuchendmeldung" (notification of job search). This will allow the agency to provide you with job placement assistance and potentially prevent or shorten your unemployment period. This notification should be made as early as possible.

2. Arbeitslosmeldung: Once your unemployment has started or is about to start, you need to make an "Arbeitslosmeldung" (notification of unemployment). This is a prerequisite for receiving unemployment benefits. You can make this notification online through the Fachportal der Bundesagentur für Arbeit (Federal Employment Agency's specialized portal) or in person at your local Agentur für Arbeit office.

3. Arbeitslosengeld: By making the Arbeitslosmeldung, you are also simultaneously applying for Arbeitslosengeld (unem

In [272]:
result

'If you are facing unemployment next month, you should take the following steps:\n\n1. Arbeitsuchendmeldung: You should inform the Agentur für Arbeit (Federal Employment Agency) about your impending unemployment by making an "Arbeitsuchendmeldung" (notification of job search). This will allow the agency to provide you with job placement assistance and potentially prevent or shorten your unemployment period. This notification should be made as early as possible.\n\n2. Arbeitslosmeldung: Once your unemployment has started or is about to start, you need to make an "Arbeitslosmeldung" (notification of unemployment). This is a prerequisite for receiving unemployment benefits. You can make this notification online through the Fachportal der Bundesagentur für Arbeit (Federal Employment Agency\'s specialized portal) or in person at your local Agentur für Arbeit office.\n\n3. Arbeitslosengeld: By making the Arbeitslosmeldung, you are also simultaneously applying for Arbeitslosengeld (unemployme

In [ ]:
for doc in rresponse[1]:
    page_content = doc.page_content
    metadata = doc.metadata
    source = metadata['source']
    title = metadata['title']
    
    # Process or use the extracted information as needed
    print(f"Source: {source}")
    print(f"Title: {title}")
    print(f"Page Content: {page_content}")
    print("-----")

# Agent with Tools - openai cookbook
https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain

## imports

In [19]:
from typing import List, Union
import re
import openai
import pandas as pd
from langchain import LLMChain, OpenAI, SerpAPIWrapper

# Langchain imports
from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import RetrievalQA

# LLM wrapper
from langchain.chat_models import ChatOpenAI

# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings

# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
from langchain.vectorstores import Pinecone
from tqdm.auto import tqdm

## template with history

In [14]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are BureauBot, a professional search engine who provides informative answers to users. Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

## template without history

In [26]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

## custom prompt template

In [27]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## custom output parser

In [28]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [29]:
llm = OpenAI(temperature=0)

## Index tool from documents in vector store
storage_context = StorageContext.from_defaults(persist_dir="vector_db")
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine(
    streaming=True, 
    text_qa_template=qa_template, 
    similarity_top_k=5
)

tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(query_engine.query(q)),
        description="useful for when you want to answer questions about work and unemployment. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
    Tool(
        name="Websearch",
        func=lambda q: str(qa_chain({"question": q})),
        description="useful for when you want to answer questions about work and unemployment and the LlamaIndex Tool does not have a sufficient answer in it's documents. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [30]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)

multi_tool_executor = AgentExecutor.from_agent_and_tools(
    agent=multi_tool_agent, tools=tools, verbose=True, memory=multi_tool_memory
)

In [31]:
multi_tool_executor.run("Where can I find information about the duties of landlords?")



> Entering new AgentExecutor chain...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"

Thought: I should look for information about the duties of landlords
Action: Websearch
Action Input: duties of landlordsINFO:langchain.retrievers.web_research:Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
Generating questions for Google Search ...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4295 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

# Tools as Classes

In [ ]:
import logging
import os

from bot_utils import default_engine
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from llama_index import Prompt
from llama_index.langchain_helpers.agents import IndexToolConfig, LlamaIndexTool
from langchain.tools import BaseTool
from langchain.agents import tool

# logging to see what sites and documents the web retriever is using
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# Search
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

search = GoogleSearchAPIWrapper()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


class EngineTool:
    def __init__(self):
        
        self.qa_template = Prompt(
            "We have provided context information below. \n"
            "---------------------\n"
            "{context_str}"
            "\n---------------------\n"
            "Do not give me an answer if it is not mentioned in the context as a fact. \n"
            "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
        )

        self.folder_with_index = "vector_db"
        self.number_top_results = 5

        self.default_engine = default_engine(
            self.folder_with_index, self.qa_template, self.number_top_results
        )

        # ERROR: IndexToolConfig awaits an instance of BaseQueryEngine
        self.tool_config = IndexToolConfig(
            query_engine=default_engine,
            name=f"Vector Index",
            description=f"Useful for when you want to answer queries about work and unemployment in Berlin, Germany",
            tool_kwargs={"return_direct": True},
        )

    def run(self):
        return LlamaIndexTool.from_tool_config(self.tool_config)


class RetrieverTool:
    """Tool to find information about work, unemployment, laws and adminstrative infromation in Berlin, Germany"""

    def __init__(self):
        # Initialize LLM
        self.llm = llm

        # Vectorstore
        self.vectorstore = Chroma(
            embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
        )

        # Initialize
        self.web_research_retriever = WebResearchRetriever.from_llm(
            vectorstore=self.vectorstore,
            llm=self.llm,
            search=search,
        )

    def run(self, user_input):
        qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
            self.llm, retriever=self.web_research_retriever
        )

        return qa_chain({"question": user_input})


class ToolChainAgent:
    def __init__(self):
        self.llm = llm

        # initialize tools
        self.q_engine_fct = EngineTool().run()
        self.retriever_fct = RetrieverTool(self.llm).run

        self.tools = [
            # embedding first
            Tool(
                name="query engine",
                description="use this tool first, to get anwsers about work and unemployment in Berlin, Germany",
                func=self.q_engine_fct,
            ),
            # web retriever goes through programmable search engine
            Tool(
                name="web_retriever",
                description="use this tool when the first tool failed and you want to answer questions about work and unempoyment in Berlin, Germany",
                func=self.retriever_fct,
            ),
        ]

        self.agent = initialize_agent(
            self.tools,
            self.llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
        )

    def run(self, user_input):
        """
        Agent runs query with the listed tools:

        1. query engine from custom embedding
        2. web retriever, which goes through programmable google-search

        Run the agent instance like this:
        agent = ToolChainAgent()
        agent.run(promt)
        """
        return self.agent(user_input)
